In [1]:
!pip install transformers accelerate torch pillow

import torch
print(torch.cuda.is_available())
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image



     |████████████████████████████████| 330 kB 1.3 MB/s eta 0:00:01
True


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.8/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.8/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The to

In [2]:

# Load model and processor (this will download weights from Hugging Face Hub)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    torch_dtype=torch.float16,
    device_map="auto"
)



KeyboardInterrupt: 

In [ ]:
print(model.device)

cuda:0


In [ ]:
# Example image
image = Image.open("example.jpg").convert("RGB")
prompt = "Describe the image Answer:"

# Preprocess input
inputs = processor(images=image, text=prompt, return_tensors="pt").to(model.device)

# Generate output
generated_ids = model.generate(**inputs, max_new_tokens=50)

In [ ]:
generated_text = processor.decode(generated_ids[0], skip_special_tokens=True)

print(generated_text)

 A man is kite surfing



In [ ]:
for name, module in model.named_children():
    print(name, ":", module._get_name())
    # Get public methods
    public_methods = [m for m in dir(module) if callable(getattr(module, m)) and m.__contains__("forward") ]
    print("  Public methods:", public_methods)
    print()

/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


vision_model : Blip2VisionModel
  Public methods: ['_hook_rss_memory_post_forward', '_hook_rss_memory_pre_forward', '_old_forward', '_slow_forward', 'forward', 'forward', 'register_forward_hook', 'register_forward_pre_hook']

qformer : Blip2QFormerModel
  Public methods: ['_hook_rss_memory_post_forward', '_hook_rss_memory_pre_forward', '_old_forward', '_slow_forward', 'forward', 'forward', 'register_forward_hook', 'register_forward_pre_hook']

language_projection : Linear
  Public methods: ['_old_forward', '_slow_forward', 'forward', 'forward', 'register_forward_hook', 'register_forward_pre_hook']

language_model : OPTForCausalLM
  Public methods: ['_hook_rss_memory_post_forward', '_hook_rss_memory_pre_forward', '_slow_forward', 'forward', 'register_forward_hook', 'register_forward_pre_hook']



In [ ]:
image =  torch.randn(1, 3, 224, 224).to(model.device)
y = (model.vision_model(image))
print(y.last_hidden_state.shape)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 7.75 GiB total capacity; 4.69 GiB already allocated; 53.00 MiB free; 4.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print(model.qformer)